In [1]:
import requests
import pandas as pd

pd.set_option('display.max_columns', None)


In [2]:
import requests
import pandas as pd

pd.set_option('display.max_columns', None)

station_names = ["Paris Bercy", "Paris Saint-Lazare", "Paris Gare de Lyon", "Paris Gare du Nord", "Paris Montparnasse", "Paris Est", "Paris Austerlitz"]
records = []

for name in station_names:
    for year in range(2019, 2023):
        url = f"https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=Paris&rows=10000&sort=date&facet=date&facet=gc_obo_gare_origine_r_name&facet=gc_obo_type_c&refine.gc_obo_gare_origine_r_name={name.replace(' ', '+')}&refine.date={year}"
        response = requests.get(url)
        data = response.json()
        records += data['records']

df = pd.json_normalize(records)
df = df.loc[:, ['fields.date', 'fields.gc_obo_gare_origine_r_name', 'fields.gc_obo_type_c']]
df = df.rename(columns={'fields.date': 'date', 'fields.gc_obo_gare_origine_r_name': 'gare', 'fields.gc_obo_type_c': 'type d\'objet'})
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

# création du CSV. 
df.to_csv("objets-trouves.csv", index=False)

df

,date,gare,type d'objet
0,2019-12-29,Paris Bercy,"Porte-monnaie / portefeuille, argent, titres"
1,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
2,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
3,2019-12-26,Paris Bercy,"Bagagerie: sacs, valises, cartables"
4,2019-12-26,Paris Bercy,"Vêtements, chaussures"
...,...,...,...
81627,2022-01-04,Paris Austerlitz,"Porte-monnaie / portefeuille, argent, titres"
81628,2022-01-04,Paris Austerlitz,"Porte-monnaie / portefeuille, argent, titres"
81629,2022-01-04,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"
81630,2022-01-04,Paris Austerlitz,"Bagagerie: sacs, valises, cartables"


In [3]:
import sqlite3
import requests
import pandas as pd

# se connecter à la base de données SQLite
connexion = sqlite3.connect("db_objets_trouves.db")
curseur = connexion.cursor()

# créer la table pour stocker les données
curseur.execute("""
    CREATE TABLE IF NOT EXISTS objets_trouves (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT NOT NULL,
        gare TEXT NOT NULL,
        type_objet TEXT NOT NULL
    )
""")


# sauvegarder les modifications et fermer la connexion
connexion.commit()
connexion.close()



In [4]:

import sqlite3
import pandas as pd

# se connecter à la base de données SQLite
connexion = sqlite3.connect("db_objets_trouves.db")
curseur = connexion.cursor()

# lire le fichier CSV dans un DataFrame pandas
df = pd.read_csv("objets-trouves.csv")

# insérer les données dans la table objets_trouves
for index, row in df.iterrows():
    curseur.execute("INSERT INTO objets_trouves (date, gare, type_objet) VALUES (?, ?, ?)", (row["date"], row["gare"], row["type d'objet"]))

# sauvegarder les modifications et fermer la connexion
connexion.commit()
connexion.close()
